In [2]:
import pandas as pd
import numpy as np
import re
from pprint import pprint
from datetime import datetime

#scraping
import GetOldTweets3 as got
import time

#plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
def get_tweets(text_query, start_date, end_date, lang, location, within):
   
    # specifying tweet search criteria 
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(text_query)\
                          .setSince(start_date)\
                          .setUntil(end_date)\
                          .setLang(lang)\
                          .setNear(location)\
                          .setWithin(within)\
                          .setMaxTweets(1000)#set max tweets no.of tweets u may retrieve
    # scraping tweets based on criteria
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    
    # creating list of tweets with the tweet attributes 
    # specified in the list comprehension
    text_tweets = [[tw.date,
                tw.username,
                tw.text,
                tw.retweets,
                tw.favorites,
                tw.hashtags,
                tw.permalink] for tw in tweet]
    
    # creating dataframe, assigning column names to list of
    # tweets corresponding to tweet attributes
    tw_df = pd.DataFrame(text_tweets, 
                         columns = ['createdat','Username', 'Text', 'Retweets', 'Favorites', 'HashTags','source'])
    
    return tw_df


In [3]:
text_queries = ['hopeless','depression','phobia','anxiety','stress','suicide','shame','grief','ocd','fear','insominia','ptsd','anorexia','wfh ,suicideawarness','suicideprevention','suicidal'] 
start_date = '2019-07-01'
end_date = '2020-07-20'
lang = 'en'
within = '150mi'
maxtweet = 1000
#specify cities
citys = ['New York City, New York','Boston, Massachusetts','Chicago, Illinois','Detroit, Michigan','Los Angeles, California','Houston, Texas','Newark, New Jersey','Miami, Florida','Philadelphia, Pennsylvania','New Orleans, Louisiana']

In [4]:
import time

c = 0
for text_query in text_queries:
    c += 1
    cities = []
    for city in citys:
        location = city
        cities.append(city)
        df = get_tweets(text_query, start_date, end_date, lang, location, within)
        df.to_csv(city + '.csv', index = False)
        time.sleep(150)#wait for escaping twitter rate limit
    city_column = ['NY', 'BOSTON', 'CHI', 'DETROIT', 'LA', 'HOUSTON', 'NEWARK', 'MIA', 'PHIL', 'NEW ORLEANS']
    cities_df = pd.DataFrame()
    time.sleep(10)
    for i in range(len(city_column)):
        df = pd.read_csv(cities[i] + '.csv')
        df['City'] = city_column[i]
        cities_df = pd.concat([dep_anx, df], axis = 0)
    new = 'new' + str(c) + '.csv'
    cities_df.to_csv(new, index = False)
final_df = pd.DataFrame()
for i in range(len(text_queries)):
    st = 'new' + str(i+1) + '.csv'
    df = pd.read_csv(st)
    df['keyword'] = text_queries[i]
    final_df = pd.concat([final_df,df],axis=0)
txt = '_'.join(text_queries)+ '.csv'
final_df.to_csv(txt)

In [5]:
final_df.head()

,createdat,Username,Text,Retweets,Favorites,HashTags,source,City,keyword
0,2020-07-19 23:59:13+00:00,hamsinthewest,Dibs on Fresh Baby Witches as a band name.,0,1,NaN,https://twitter.com/hamsinthewest/status/12850...,NY,hopeless
1,2020-07-19 23:52:47+00:00,RobertTrombett4,Thanks!,0,0,NaN,https://twitter.com/RobertTrombett4/status/128...,NY,hopeless
2,2020-07-19 23:52:20+00:00,RobertTrombett4,He always sounds exactly like somebody who kno...,0,0,NaN,https://twitter.com/RobertTrombett4/status/128...,NY,hopeless
3,2020-07-19 23:51:02+00:00,MelBoomBos,We do. There's lots of hopeless people like me...,0,1,NaN,https://twitter.com/MelBoomBos/status/12849992...,NY,hopeless
4,2020-07-19 23:49:50+00:00,RobertTrombett4,"Like, probably, Mr. Royal Rose up there?",0,0,NaN,https://twitter.com/RobertTrombett4/status/128...,NY,hopeless


In [6]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91604 entries, 0 to 3774
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   createdat  91604 non-null  object
 1   Username   91604 non-null  object
 2   Text       91604 non-null  object
 3   Retweets   91604 non-null  object
 4   Favorites  91604 non-null  object
 5   HashTags   11700 non-null  object
 6   source     91604 non-null  object
 7   City       91604 non-null  object
 8   keyword    91604 non-null  object
dtypes: object(9)
memory usage: 7.0+ MB


In [7]:
final_df['keyword'].value_counts()

anxiety              10000
fear                 10000
stress               10000
depression            9703
shame                 9558
suicide               8871
ptsd                  6836
grief                 6455
insominia             5406
hopeless              4577
suicidal              3775
ocd                   2827
suicideprevention     1734
phobia                1465
anorexia               397
Name: keyword, dtype: int64

In [9]:
print('shape:', final_df.shape)
print('number of tweets:', final_df['Text'].nunique())
print('number of users:', final_df['Username'].nunique())

print("range:", final_df['createdat'].min(),",", final_df['createdat'].max())
print("dates:",final_df['createdat'].nunique())

shape: (91604, 9)
number of tweets: 76257
number of users: 47629
range: 2019-07-01 00:33:37+00:00 , 2020-07-19 23:59:54+00:00
dates: 76362
